# Final porject
## Week 3 task 2-- Import Geo-data for Neighborhoods in Toronto

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup

### Read data from Wikipedia
We assume the first table in the webpage is the table we want

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)
dfs=pd.read_html(results.text)
df=dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Clean dataset


Remove unassigned postal code.

In [3]:
# remove rows with Borough = Not assigned
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Combine entries with the same postal code.

In [4]:
# combine the data with same postcode
d = {'Borough': 'first', 'Neighbourhood': ', '.join}
df= df.groupby('Postal Code', as_index = False).aggregate(d).reindex(columns=df.columns)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Use borough name if neighbourhood name is not given. *(Assuming the borough itself is a neighbourhood)* 

In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
df.shape

(103, 3)

## Import geo-data

Read geodata stored in the csv file

In [7]:
# import data
path='data/'
file_name='Geospatial_Coordinates.csv'
df_geo=pd.read_csv(path+file_name)
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two datasets base on postcode

In [8]:
df_merged = pd.merge(df,df_geo, on = 'Postal Code',how='outer')
print('df_merged.shape=', df_merged.shape)
df_merged.head()

df_merged.shape= (103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Work only on the data with 'Toronto' in the borough name

In [9]:
df_reduced=df_merged[df_merged['Borough'].str.contains("Toronto")]
print('df_reduced.shape=', df_reduced.shape)
df_reduced.head()

df_reduced.shape= (39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
